<a href="https://www.kaggle.com/code/dorbezfradj/ship-detection-ps1?scriptVersionId=264047775" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# importation des bibliothéques
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np 
from skimage.util import montage
from skimage.segmentation import mark_boundaries
import tensorflow as tf

In [ ]:
train_image_dir = "/kaggle/input/airbus-ship-detection/train_v2"
train_encode_file = "/kaggle/input/airbus-ship-detection/train_ship_segmentations_v2.csv"
test_image_dir= "/kaggle/input/airbus-ship-detection/test_v2"

In [ ]:
#Le répertoire du train
train_images = os.listdir(train_image_dir)
print(f"Total  {len(train_images)} images dans le répertoire du train . \nvoici les 5 premières images  : - {train_images[:5]}   ")

In [ ]:
#Le répertoire du test
test_images = os.listdir(test_image_dir)
print(f"Total  {len(test_images)} images dans le répertoire du test . \nvoici les 5 premières images  : - {test_images[:5]}   ")

In [ ]:
# visualiser quelques images de tests
plt.figure(figsize=(15,15))
for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow(cv2.imread(test_image_dir+ '/' +test_images[i]))
    plt.title(f"{test_images[i]}", weight='bold')
    plt.axis('off')
plt.tight_layout()

In [ ]:
df = pd.read_csv(train_encode_file)

In [ ]:
df.info() #qlq infomrations sur train_encode_file

In [ ]:
df.describe()

In [ ]:
df.head(10)

In [ ]:
df[df["ImageId"]=="00021ddc3.jpg"] # dans df, dans la même image nous avons plus d'un navire et chaque navire est rangée dans une ligne de notre table df

In [ ]:
# construire un dictionnaire dont la clé est l'id de l'image et la valeur est le nombre des navires présents dans l'image
ship_num = {}

for index, ligne in df.iterrows():
    id_image = ligne['ImageId']
    encoding = ligne['EncodedPixels']
    
    if pd.isna(encoding) or encoding == "":
        ship_num[id_image] = 0
    else:
        if id_image in ship_num:
            ship_num[id_image] += 1
        else:
            ship_num[id_image] = 1

print(ship_num)

In [ ]:
img=cv2.imread(train_image_dir+"/30d3f7721.jpg")
plt.imshow(img)
print(f"nombres de navires dans cette image est : {ship_num['30d3f7721.jpg'] }") #on verifie le nombre de navire pour l'image d'id 30d3f7721.jpg

In [ ]:
print(len(ship_num))
#coinside avec le nombre unique des element 
# nombre des images

In [ ]:
# Transformez le dictionnaire en DataFrame pour faciliter la manipulation des informations
MonData = pd.DataFrame(list(ship_num.items()), columns=['ImageId', 'ships'])

In [ ]:
MonData.head(10)

In [ ]:
#on a verifier que l'image dont l'id est 00003e153.jpg ne presente pas des navires
img=cv2.imread(train_image_dir +"/00003e153.jpg")
plt.imshow(img)
print("cette image d'id \"00003e153.jpg\" ne represente aucune navire d'aprés le tableau précédent.")

In [ ]:
plt.figure(figsize=(12, 8)) # Ajuster la taille de la figure
sns.countplot(data=MonData, x ='ships', palette='Set2')
# Ajouter des labels et un titre
plt.title('Distribution du nombre d\'images', fontsize=16)
plt.xlabel('Nombre des navires', fontsize=14)
plt.ylabel('Nombre d\'occurrences', fontsize=14)
plt.show()

In [ ]:
MonData['ships'].value_counts() # nombre des image qui ont le meme nombre des navires
#On peut observer un déséquilibre dans la répartition des images en fonction du nombre de navires.

In [ ]:
# list of image paths
image_paths = [
    os.path.join(train_image_dir, filename) for filename in os.listdir(train_image_dir)
]

In [ ]:
image_paths[10:20]

In [ ]:
print(df.isnull().sum()) # Vérifier s'il y a des valeurs manquantes

In [ ]:
# Analyse détaillée des images
for i in range(5):
    image = cv2.imread(image_paths[i])
    hauteur, largeur, canaux = image.shape  # Obtenir les dimensions de l'image
    taille_image = hauteur * largeur       # Calculer la taille de l'image
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(f"Image {i + 1}")
    plt.axis('off')
    plt.show()
    print(f"Dimensions de l'image : {hauteur} x {largeur} pixels")
    print(f"Nombre de canaux : {canaux}")
    print(f"Taille de l'image : {taille_image} pixels")

In [ ]:
print('nombre des images dans csv ',len(df))
print('nombre des images dans fichier image ',len(image_paths))

In [ ]:
nombre_images_sans_navire = 0
nombre_images_avec_navire = 0

# Parcourez le dictionnaire ship_num
for nombre_navires in ship_num.values():
    if nombre_navires == 0:
        nombre_images_sans_navire += 1
    else:
        nombre_images_avec_navire += 1

# Créez deux listes pour les abscisses et les ordonnées du graphique
categories = ['Sans Navire', 'Avec Navire']
nombre_images = [nombre_images_sans_navire, nombre_images_avec_navire]
print("le nombre des images sans navire",nombre_images_sans_navire)
print("le nombre des images avec navire",nombre_images_avec_navire)
# Créez un graphique à barres
plt.bar(categories, nombre_images)

# Étiquetez les axes du graphique
plt.xlabel('Catégorie')
plt.ylabel('Nombre d\'images')

# Affichez le graphique
plt.show()

In [ ]:
# Créer une nouvelle colonne 'Size' qui contient le nombre de pixels par navire
df['Size'] = df['EncodedPixels'].apply(lambda x: 0 if pd.isna(x) else sum(map(int, str(x).split()[1::2])))

# Grouper les données par taille et compter le nombre de navires de chaque taille
tailles_navires = df[df['Size'] > 0]['Size'].value_counts().reset_index()
tailles_navires.columns = ['Taille du Navire (en pixels)', 'Nombre de Navires']

# Trier les données par taille croissante
tailles_navires = tailles_navires.sort_values(by='Taille du Navire (en pixels)')

# Tracer le graphique
plt.figure(figsize=(12, 6))
plt.hist(tailles_navires['Taille du Navire (en pixels)'], bins=50, color='skyblue', edgecolor='black')
#plt.bar(tailles_navires['Taille du Navire (en pixels)'], tailles_navires['Nombre de Navires'])
plt.title('Nombre de Navires en fonction de la Taille')
plt.xlabel('Taille du Navire (en pixels)')
plt.ylabel('Nombre de Navires')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
df['file_size_kb']=df['ImageId'].map(lambda c_img_id:os.stat(os.path.join(train_image_dir,c_img_id)).st_size/1024) # calcul de taille des fichiers


In [ ]:
df.head()

In [ ]:
def plot_color_histogram(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convertir l'image de BGR à RGB (Matplotlib utilise le format RGB)
    plt.imshow(image_rgb)
    plt.title('Image originale')
    plt.axis('off')
    plt.show()
    r, g, b = cv2.split(image_rgb)  # Diviser l'image en ses canaux de couleur (Rouge, Vert, Bleu)
    plt.figure(figsize=(12, 4))
    # Histogramme pour le canal rouge
    plt.subplot(131)
    plt.hist(r.flatten(), bins=256, color='red', alpha=0.7, rwidth=0.8)
    plt.title('Histogramme Rouge')
    plt.xlabel('Valeur de pixel')
    plt.ylabel('Fréquence')

    # Histogramme pour le canal vert
    plt.subplot(132)
    plt.hist(g.flatten(), bins=256, color='green', alpha=0.7, rwidth=0.8)
    plt.title('Histogramme Vert')
    plt.xlabel('Valeur de pixel')
    plt.ylabel('Fréquence')

    # Histogramme pour le canal bleu
    plt.subplot(133)
    plt.hist(b.flatten(), bins=256, color='blue', alpha=0.7, rwidth=0.8)
    plt.title('Histogramme Bleu')
    plt.xlabel('Valeur de pixel')
    plt.ylabel('Fréquence')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_color_histogram(image_paths[0])
plot_color_histogram(image_paths[10])
plot_color_histogram(image_paths[900])

In [ ]:
def orientation_distribution(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Charger l'image en niveaux de gris
    # Appliquer le filtre de Sobel pour obtenir les gradients
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=5)
    
    gradient_orientation = np.arctan2(sobely, sobelx) # Calculer l'orientation des gradients
    gradient_orientation_degrees = np.degrees(gradient_orientation) # Convertir l'orientation en degrés
    flattened_orientation = gradient_orientation_degrees.flatten()  # Aplatir l'array pour l'histogramme
    # Afficher l'image originale
    plt.subplot(121)
    plt.imshow(image, cmap='gray')
    plt.title('Image originale')
    plt.axis('off')
    # Afficher l'histogramme des orientations
    plt.subplot(122)
    plt.hist(flattened_orientation, bins=36, range=[-180, 180], color='black', alpha=0.7)
    plt.title('Histogramme des orientations')
    plt.xlabel('Orientation en degrés')
    plt.ylabel('Fréquence')
    plt.tight_layout()  # Ajuster automatiquement les espaces pour éviter le chevauchement
    plt.show()

In [ ]:
orientation_distribution(image_paths[10])

**la fréquence maximale du diagramme est de 17500, ce qui correspond à une orientation de 100 degrés. Cela suggère que l'orientation dominante des objets dans l'image est de 100 degrés, qui correspond a l'orientation du navire**

In [ ]:
for i in range (1,5):
    orientation_distribution(image_paths[i])

In [ ]:
def detect_keypoints(image_path):
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create() # Initialiser l'extracteur ORB
    keypoints, descriptors = orb.detectAndCompute(gray_image, None)  # Détecter les points d'intérêt et les descripteurs avec ORB
    image_with_keypoints = cv2.drawKeypoints(image, keypoints, None) # Dessiner les points d'intérêt sur l'image
    # Afficher l'image avec les points d'intérêt
    plt.imshow(cv2.cvtColor(image_with_keypoints, cv2.COLOR_BGR2RGB))
    plt.title('Détection de points d\'intérêt avec ORB (Oriented FAST and Rotated BRIEF)')
    plt.axis('off')
    plt.show()

In [ ]:
detect_keypoints(image_paths[10])

**on remarque que la plupart des points d'intérêt obtenus par le filtre ORB est sur le navire**

In [ ]:
def evaluate_sharpness_contrast(image_path):
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplacian = cv2.Laplacian(gray_image, cv2.CV_64F) # Calculer le filtre Laplacien pour évaluer la netteté
    sharpness = np.var(laplacian) # Calculer la variance de l'image Laplacienne pour la netteté
    contrast = np.max(gray_image) - np.min(gray_image) # Calculer le contraste de l'image
    # Normaliser les valeurs entre 0 et 1
    sharpness_normalized = sharpness / (sharpness + contrast)
    contrast_normalized = contrast / (sharpness + contrast)
    # Produire un score global normalisé
    score_normalized = (sharpness_normalized + contrast_normalized) / 2
    return score_normalized

**on a 192 225 exemples, analyser toutes les images serait trop coûteux en temps d'exécution. Nous nous concentrons donc sur un échantillon de 4000 images pour maintenir l'efficacité de la fonction evaluate_sharpness_contrast et le bloc de code qui calcule le shape**

In [ ]:
contrast= set() # initialisation d'un ensemble 
for i in range(4000):
    pt = image_paths[i]
    score = evaluate_sharpness_contrast(pt)
    contrast.add(score)

In [ ]:
contrast

**on remarque que toutes les images ont le même score de contraste et de netteté**

In [ ]:
shape_unique = set() 
for i in range(4000):
    pt = image_paths[i]
    image = cv2.imread(image_paths[i]) 
    shape_unique.add(image.shape)

In [ ]:
shape_unique

**L'EDA nous a fourni une compréhension initiale des caractéristiques des images.Maintenant, nous concentrerons sur les techniques spécifiques de prétraitement des données, y compris la segmentation par un mask binaire, pour extraire des informations plus fines sur les régions d'intérêt vu que La segmentation permettra une détection plus précise des navires surtout dans une distribution uniforme de l'eau ainsi que les positions des navires sont données dans le fichier Csv. Nous explorerons également la sélection du modèle et l'optimisation des hyperparamètres, qui va être pour le moment un U-Net et n'oublions pas l'augmentation des données**

# **Prétraitement des données**

In [ ]:
def rle_decode(mask_rle, shape=(768, 768)):

    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  

def masks_image(in_mask_list):
    # Prendre les masques de navire individuels et créer un tableau de masques unique pour tous les navires
    all_masks = np.zeros((768, 768), dtype = np.uint8)
    for mask in in_mask_list:
        if isinstance(mask, str):
            all_masks |= rle_decode(mask)
    return all_masks

In [ ]:
# afficher une image d'entainement et sa masque
from skimage.io import imread
for num in [100, 88, 5]:
    rle_0 = df.query(f'ImageId=="{train_images[num-1]}"')['EncodedPixels']
    img_0 = masks_image(rle_0)
    original = imread(train_image_dir+"/"+train_images[num-1])
    plt.figure(figsize=(15, 8))
    plt.subplot(1, 2, 1)
    plt.title(f"Original - Train Image {original.shape}")
    plt.imshow(original)
    plt.subplot(1, 2, 2)
    plt.title(f"Mask generated from the RLE data for each ship {img_0.shape}")
    plt.imshow(img_0, cmap = "Blues_r")
    plt.tight_layout()
    plt.show()

In [ ]:
MonData['file_size_kb']=MonData['ImageId'].map(lambda c_img_id:os.stat(os.path.join(train_image_dir,c_img_id)).st_size/1024) # calcul de taille des fichiers
MonData['file_size_kb'].hist()

In [ ]:
# Filtrer les données pour inclure uniquement les entrées dont la taille de fichier est supérieure à 50 ko
MonData = MonData[MonData.file_size_kb > 50] # Tracer un histogramme de la colonne 'file_size_kb' pour visualiser la distribution des tailles de fichier
MonData['file_size_kb'].hist()
MonData.sample(7) # Sélectionner aléatoirement 7 échantillons de l'ensemble de données
MonData.shape
df.drop(['Size'], axis=1, inplace=True) # Supprimer la colonne 'Size' du DataFrame df 


In [ ]:
MonData.head()

In [ ]:
from sklearn.model_selection import train_test_split                   
# Diviser l'ensemble de données 'MonData' en ensembles d'entraînement (train) et de validation (valid)
train, valid = train_test_split(MonData, test_size=0.2, stratify=MonData['ships'])
print("le shape du dataframe de train est :",train.shape)


In [ ]:
train_df = pd.merge(df, train) # Fusionner le DataFrame 'df' avec le DataFrame 'train' 
train_df.sort_values(by='ImageId') # Trier le DataFrame résultant 'train_df' par la colonne 'ImageId'
# Afficher le DataFrame 'train_df'
print(train_df)


**le nombre des lignes de train était 153031 après la division en train et validation mais après le fusionnage la taille de train df à 184071 lignes qui présentent des lignes de la même image mais de navire différent car les lignes de Df représentent des navires et une image peut continuer plusieurs navires**

In [ ]:
valid_df = pd.merge(df, valid)
print("le shape du dataframe de validation est:",valid_df.shape)

In [ ]:
print("on trouve:")
print(train_df.shape[0], 'training masks,')
print(valid_df.shape[0], 'validation masks.')

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=train_df, x='ships', palette='Set2')
plt.show()

In [ ]:
# notre but dans ce code et d'essayer de fusionner le nombre des navires par image en deux classes pour balances les différentes images
train_df['grouped_ship_count'] = train_df.ships.map(lambda x: (x+1)//2).clip(0,7)
train_df.head()

In [ ]:
# 0 est l'image ne represente aucun navire
# 1 les images qui presentent un navire et les images qui presentent 2 navires 
train_df.grouped_ship_count.value_counts() # Afficher le décompte des valeurs uniques dans la colonne 'grouped_ship_count' du DataFrame 'train_df'

In [ ]:
# Fonction pour échantillonner des entrées en fonction du nombre de navires 
def sample_ships(in_df, base_rep_val=2100):
    if in_df['ships'].values[0] == 0: # Vérifier si le nombre de navires est égal à 0
        return in_df.sample(base_rep_val // 3)   # échantillonner un tiers de la valeur 2100 
    else:
        return in_df.sample(base_rep_val)


In [ ]:
balanced_train_df = train_df.groupby('grouped_ship_count').apply(sample_ships) # créer un dataframe plus ou moins balancé
balanced_train_df.grouped_ship_count.value_counts()
print("le shape du dataframe equiblibré est :",balanced_train_df.shape)

In [ ]:
#pour chaque sous groupe de 0a 7 on va afficher le nombre des image de train groupeés par le nombre des navires
for i in range(8):
    df_val_counts = balanced_train_df[balanced_train_df.grouped_ship_count==i].ships.value_counts()
    print(f"Data frame for grouped ship count = {i}:-\n{df_val_counts}\nSum of Values:- {df_val_counts.values.sum()}\n\n")

In [ ]:
#Visualiser la distubution des données de training avant et après l'équilibrage
plt.figure(figsize=(15, 5))
plt.suptitle("Train Data", fontsize=18, color='r', weight='bold')

plt.subplot(1, 2, 1)
sns.countplot(data=train_df, x='ships', palette='Set2')
plt.title("Ship Counts - Before Balancing", fontsize=15)
plt.ylabel("Count", fontsize=13)
plt.xlabel("# Ships in an image", fontsize=13)

plt.subplot(1, 2, 2)
sns.countplot(data=balanced_train_df, x='ships', palette='Set2')
plt.title("Ship Counts - After Balancing", fontsize=15)
plt.xlabel("# Ships in an image", fontsize=13)
plt.ylabel("Count", fontsize=13)

plt.tight_layout()
plt.show()

In [ ]:
BATCH_SIZE = 48 #taille du chaque batch
IMG_SCALING = (3, 3) #parametre de rédimensionement de l'image
def make_image_gen(in_df, batch_size=BATCH_SIZE):
    all_batches = list(in_df.groupby('ImageId'))
    out_rgb = []
    out_mask = []
    while True: 
        np.random.shuffle(all_batches) # Mélanger l'ordre des lots(batch)
        for c_img_id, c_masks in all_batches:
            # Chemin vers l'image RGB
            rgb_path = os.path.join(train_image_dir, c_img_id)
            c_img = imread(rgb_path) # Charger l'image
            c_mask = np.expand_dims(masks_image(c_masks['EncodedPixels'].values), -1) # Créer le masque à partir des encodages de pixels
            # Redimensionner l'image et le masque si spécifié
            if IMG_SCALING is not None:
                c_img = c_img[::IMG_SCALING[0], ::IMG_SCALING[1]]
                c_mask = c_mask[::IMG_SCALING[0], ::IMG_SCALING[1]]
            
            # Ajouter l'image et le masque aux listes
            out_rgb += [c_img]
            out_mask += [c_mask]            
            # Si la taille du lot (batch) est atteinte, renvoyer le lot
            if len(out_rgb) >= batch_size:
                yield np.stack(out_rgb, 0) / 255.0, np.stack(out_mask, 0)
                out_rgb, out_mask = [], []  # Réinitialiser les listes pour le prochain lot

In [ ]:
train_gen = make_image_gen(balanced_train_df) #appel de la fonction 
# Image et Mask
train_x, train_y = next(train_gen)
print(f"train_x ~\nShape: {train_x.shape}\nvaleur min: {train_x.min()}\nvaleur max: {train_x.max()}")
print(f"\ntrain_y ~\nShape: {train_y.shape}\nvaleur min: {train_y.min()}\nvaleur max: {train_y.max()}")

In [ ]:
# Visualisation d'un lot d'entraînement
from skimage.segmentation import mark_boundaries
montage_rgb = lambda x: np.stack([montage(x[:, :, :, i]) for i in range(x.shape[3])], -1)
batch_rgb = montage_rgb(train_x) # Créer un montage d'images a partir de pile
batch_seg = montage(train_y[:, :,:,0])  # Créer un montage de masques
batch_overlap = mark_boundaries(batch_rgb, batch_seg.astype(int)) # Créer des contours autour des navires dans l'image
titles = ["Images", "Segmentations", "Contours des navires dans les images"] 
colors = ['g', 'm', 'b']  
display = [batch_rgb, batch_seg, batch_overlap]   
plt.figure(figsize=(25,10))                                                        
for i in range(3):                                                                
    plt.subplot(1, 3, i+1) # Créer le sous-graphique
    plt.imshow(display[i])                                                         
    plt.title(titles[i], fontsize=18, color=colors[i])                            
    plt.axis('off')                                                                
plt.suptitle("Visualisation du Lot", fontsize=20, color='r', weight='bold') 
plt.tight_layout()                                                                 


In [ ]:
VALID_IMG_COUNT = 400 # Taille de lot validation
valid_x, valid_y = next(make_image_gen(valid_df, VALID_IMG_COUNT))
print(f"valid_x ~\nShape: {valid_x.shape}\nvaleur min: {valid_x.min()}\nvaleur max: {valid_x.max()}")
print(f"\nvalid_y ~\nShape: {valid_y.shape}\n valeur min: {valid_y.min()}\nvaleur max: {valid_y.max()}")

In [ ]:
# on va utiliser ImageDataGenerator pour l'augmentation du dnnées
from keras.preprocessing.image import ImageDataGenerator
# Paramètres pour la génération d'images augmentées
dg_args = dict(rotation_range=15,          # Plage de degrés pour les rotations aléatoires
               horizontal_flip=True,       # Effectue des retournements horizontaux aléatoires
               vertical_flip=True,         # Effectue des retournements verticaux aléatoires
               data_format='channels_last')  # channels_last se réfère à (lot, hauteur, largeur, canaux)


In [ ]:
image_gen = ImageDataGenerator(**dg_args) # Créer un générateur d'images avec les paramètres spécifiés
label_gen = ImageDataGenerator(**dg_args) # Créer un générateur d'étiquettes avec les paramètres spécifiés
# Définir une fonction pour créer un générateur augmenté
def create_aug_gen(in_gen, seed=None):
    np.random.seed(seed if seed is not None else np.random.choice(range(9999)))
    
    for in_x, in_y in in_gen:
        seed = np.random.choice(range(9999))  # definir un seed pour avoir la meme augmentation pour l'image et le mask
        g_x = image_gen.flow(255 * in_x,
                             batch_size=in_x.shape[0],
                             seed=seed,
                             shuffle=False)
        
        g_y = label_gen.flow(in_y,
                             batch_size=in_x.shape[0],
                             seed=seed,
                             shuffle=False)

        yield next(g_x) / 255.0, next(g_y) # Normaliser les valeurs des pixels de l'image et de l'étiquette


In [ ]:
# Augmenter les données d'entraînement
cur_gen = create_aug_gen(train_gen, seed=42)
t_x, t_y = next(cur_gen)

# Afficher les informations sur les données augmentées
print('x', t_x.shape, t_x.dtype, t_x.min(), t_x.max())
print('y', t_y.shape, t_y.dtype, t_y.min(), t_y.max())


In [ ]:
import random
# Nombre total d'exemples
total_examples = t_x.shape[0]
# Définir un seed pour rendre la sélection aléatoire reproductible
random_seed = 3
random.seed(random_seed)
random_indices = random.sample(range(total_examples), 4) # Sélectionner de manière aléatoire des indices pour les exemples à afficher
fig, axes = plt.subplots(2, 4, figsize=(10,6)) # Affichage de manière aléatoire des exemples d'images et de leurs masques
for i, random_index in enumerate(random_indices):
    # Afficher l'image
    axes[0, i].imshow(t_x[random_index])
    axes[0, i].set_title(f"Image {random_index + 1}")

    # Afficher le masque
    axes[1, i].imshow(t_y[random_index])
    axes[1, i].set_title(f"Masque {random_index + 1}")

plt.show()


In [ ]:
# Définir une fonction lambda pour créer un montage RGB à partir d'un tableau d'images
montage_rgb = lambda x: np.stack([montage(x[:, :, :, i]) for i in range(x.shape[3])], -1)
# Affichage final avant de passer les données au modèle
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(25, 10))

# Afficher le montage RGB des images d'entraînement
ax1.imshow(montage_rgb(t_x), cmap='gray')
ax1.set_title('Images', fontsize=18, color='g')
ax1.axis('off')

# Afficher le montage des masques
ax2.imshow(montage(t_y[:, :, :, 0]), cmap='Blues_r')
ax2.set_title('Masques', fontsize=18, color='r')
ax2.axis('off')

# Afficher les contours autour des navires dans les images d'entraînement
ax3.imshow(mark_boundaries(montage_rgb(t_x), montage(t_y[:, :, :, 0].astype(int))))
ax3.set_title('Boîte englobante', fontsize=18, color='b')
ax3.axis('off')
plt.tight_layout()


# **Construction du model**

In [ ]:
from keras import models, layers
def unet(input_size=(256,256,3)):
    inputs=layers.Input(input_size)
    c1= layers.Conv2D(16,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(inputs)
    c1=layers.Dropout(0.1)(c1)
    c1= layers.Conv2D(16,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(inputs)
    p1=layers.MaxPooling2D((2,2))(c1)
    
    c2= layers.Conv2D(32,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(p1)
    c2=layers.Dropout(0.1)(c2)
    c2= layers.Conv2D(32,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c2)
    p2=layers.MaxPooling2D((2,2))(c2)
    
    c3= layers.Conv2D(64,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(p2)
    c3=layers.Dropout(0.1)(c3)
    c3= layers.Conv2D(64,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c3)
    p3=layers.MaxPooling2D((2,2))(c3)
    
    c4= layers.Conv2D(128,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(p3)
    c4=layers.Dropout(0.1)(c4)
    c4= layers.Conv2D(128,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c4)
    p4=layers.MaxPooling2D((2,2))(c4)
    
    c5= layers.Conv2D(256,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(p4)
    c5=layers.Dropout(0.1)(c5)
    c5= layers.Conv2D(256,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c5)
    
    u6=layers.Conv2DTranspose(128,(2,2),strides=(2,2),padding='same')(c5)
    u6=layers.concatenate([u6,c4])
    c6= layers.Conv2D(128,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(u6)
    c6=layers.Dropout(0.2)(c6)
    c6= layers.Conv2D(128,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c6)
    
    u7=layers.Conv2DTranspose(64,(2,2),strides=(2,2),padding='same')(c6)
    u7=layers.concatenate([u7,c3])
    c7= layers.Conv2D(64,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(u7)
    c7=layers.Dropout(0.2)(c7)
    c7= layers.Conv2D(64,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c7)
    
    u8=layers.Conv2DTranspose(32,(2,2),strides=(2,2),padding='same')(c7)
    u8=layers.concatenate([u8,c2])
    c8= layers.Conv2D(32,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(u8)
    c8=layers.Dropout(0.2)(c8)
    c8= layers.Conv2D(32,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c8)
    
    u9=layers.Conv2DTranspose(16,(2,2),strides=(2,2),padding='same')(c8)
    u9=layers.concatenate([u9,c1])
    c9= layers.Conv2D(16,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(u9)
    c9=layers.Dropout(0.2)(c9)
    c9= layers.Conv2D(16,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c9)
    
    out = layers.Conv2D(1,(1,1),activation='sigmoid')(c9)
    
    model = models.Model(inputs=[inputs],outputs=[out])
    model.summary()
    return model

In [ ]:
nouveau_model=unet()

In [ ]:
# les differentes layer du modele
nouveau_model.layers

In [ ]:
import keras.backend as k
def dice_coeff(y_pred,y_true):
    y_true = k.cast(y_true, dtype=tf.float32)  
    y_pred = k.cast(y_pred, dtype=tf.float32)
    intersection= k.sum(y_pred * y_true,axis=[1,2,3])
    union= k.sum(y_pred,axis=[1,2,3]) + k.sum(y_true,axis=[1,2,3])
    dice_coefficient = k.mean((2. * intersection+1) / (union+1))
    return dice_coefficient 

In [ ]:
def focal_loss(y_pred,y_true, gamma=2,alpha=0.8):
    y_true = k.cast(y_true, dtype=tf.float32)  
    y_pred = k.cast(y_pred, dtype=tf.float32)
    y_pred=k.flatten(y_pred)
    y_true=k.flatten(y_true)
    BCE = k.binary_crossentropy(y_pred,y_true)
    EXP_BCE= k.exp(-BCE)
    focal_loss= k.mean(alpha * k.pow((1-EXP_BCE),gamma)* BCE)
    return focal_loss

In [ ]:
# tester la fonction qui calcule le dice_coeff
resultat_dice_coeff = dice_coeff(train_x, train_y)
print(resultat_dice_coeff.numpy())


In [ ]:
from tensorflow.keras.optimizers import Adam
nouveau_model.compile(optimizer=Adam(1e-3,beta_1=1e-6),loss=focal_loss,metrics=[dice_coeff])

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
reduce = ReduceLROnPlateau(monitor='val_dice_coeff', factor=0.23,
                                   patience=3, verbose=1, mode='max',
                                   min_delta=0.0001, cooldown=2, min_lr=1e-6)

early = EarlyStopping(monitor="val_dice_coeff", mode="max",
                      patience=20)

callbacks_list = [reduce,early]

In [ ]:
aug_gen = create_aug_gen(train_gen)


In [ ]:
nouveau_model.fit(aug_gen,epochs=200,validation_data=(valid_x, valid_y),steps_per_epoch=30,
                 callbacks=callbacks_list)

In [ ]:
nouveau_model.save('nouveau_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
final_model=load_model('nouveau_model.h5',custom_objects={'focal_loss':focal_loss,'dice_coeff':dice_coeff})

In [ ]:
def prediction(path, img_id, model):
    img = imread(os.path.join(path, img_id))
    img = img[::3, ::3] 
    img=img/255.0# Redimensionner l'image à la taille attendue par le modèle
    img = np.expand_dims(img, axis=0)

    # Assurez-vous de fournir les données à la méthode predict
    pred = model.predict(img)

    img = np.squeeze(img, axis=0)
    predo = np.squeeze(pred, axis=0)
    return img, pred

In [ ]:
# evaluation du modele
print(cv2.imread("/kaggle/input/airbus-ship-detection/test_v2/00002bd58.jpg").shape)
for sampl in range(20):
    img,pred = prediction("/kaggle/input/airbus-ship-detection/test_v2",test_images[sampl],final_model)
    fig=plt.figure(figsize=(8,8))
    fig.add_subplot(1,2,1)
    plt.imshow(img)
    plt.axis('off')
    
    pred_squeezed = np.squeeze(pred, axis=0)
    fig.add_subplot(1,2,2)
    plt.imshow(pred_squeezed)
    plt.axis('off')